# Gold standard curation: Preprocessing and single-step regression

In this stage of gold standard curation, we will do the data preprocessing, selection, and single-step regression for the 153 traits in our question set. The workflow consists of the following steps:

1. Preprocess all the cohorts related to this trait. Each cohort should be converted to a tabular form and saved to a csv file, with columns being genetic factors, the trait, and age, gender if available;
2. If there exists at least one cohort with age or gender information, conduct regression analysis with genetic features together with age or gender as the regressors.


# 1. Basic setup

In [1]:
import os
import sys
from utils import *

sys.path.append('..')
# Set your preferred name
USER = "Lisa"
# Set the data and output directories
DATA_ROOT = '/Users/wjt/Desktop/is_data'
OUTPUT_ROOT = '/Users/wjt/Desktop/is_data/v1'
TRAIT = 'Psoriatic-Arthritis'

OUTPUT_DIR = os.path.join(OUTPUT_ROOT, USER, '-'.join(TRAIT.split()))
JSON_PATH = os.path.join(OUTPUT_DIR, "cohort_info.json")
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR, exist_ok=True)

# Gene symbol normalization may take 1-2 minutes. You may set it to False for debugging.
NORMALIZE_GENE = True

In [25]:
# This cell is only for use on Google Colab. Skip it if you run your code in other environments

"""import os
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
proj_dir = '/content/drive/MyDrive/AI4Science_Public'
os.chdir(proj_dir)"""

"import os\nfrom google.colab import drive\n\ndrive.mount('/content/drive', force_remount=True)\nproj_dir = '/content/drive/MyDrive/AI4Science_Public'\nos.chdir(proj_dir)"

# 2. Data preprocessing and selection

## 2.1. The TCGA Xena dataset

In TCGA Xena, there is either zero or one cohort related to the trait. We search the names of subdirectories to see if any matches the trait. If a match is found, we directly obtain the file paths.

In [2]:
import os
from utils import *

dataset = 'TCGA'
dataset_dir = os.path.join(DATA_ROOT, dataset)
os.listdir(dataset_dir)[:]

['TCGA_Adrenocortical_Cancer_(ACC)',
 'TCGA_Bile_Duct_Cancer_(CHOL)',
 'TCGA_Lower_Grade_Glioma_(LGG)',
 'TCGA_Acute_Myeloid_Leukemia_(LAML)',
 'TCGA_Esophageal_Cancer_(ESCA)',
 'TCGA_Thyroid_Cancer_(THCA)',
 'TCGA_Ocular_melanomas_(UVM)',
 'TCGA_Ovarian_Cancer_(OV)',
 'TCGA_Prostate_Cancer_(PRAD)',
 'CrawlData.ipynb',
 'TCGA_Kidney_Papillary_Cell_Carcinoma_(KIRP)',
 'TCGA_Lung_Cancer_(LUNG)',
 'TCGA_Endometrioid_Cancer_(UCEC)',
 'TCGA_Testicular_Cancer_(TGCT)',
 'TCGA_lower_grade_glioma_and_glioblastoma_(GBMLGG)',
 'TCGA_Pancreatic_Cancer_(PAAD)',
 'TCGA_Glioblastoma_(GBM)',
 'TCGA_Melanoma_(SKCM)',
 'TCGA_Kidney_Clear_Cell_Carcinoma_(KIRC)',
 'TCGA_Mesothelioma_(MESO)',
 'TCGA_Rectal_Cancer_(READ)',
 'TCGA_Head_and_Neck_Cancer_(HNSC)',
 'TCGA_Large_Bcell_Lymphoma_(DLBC)',
 'TCGA_Uterine_Carcinosarcoma_(UCS)',
 'TCGA_Colon_Cancer_(COAD)',
 'TCGA_Breast_Cancer_(BRCA)',
 'TCGA_Bladder_Cancer_(BLCA)',
 'TCGA_Sarcoma_(SARC)',
 'TCGA_Lung_Squamous_Cell_Carcinoma_(LUSC)',
 'TCGA_Cervical_Ca

If no match is found, jump directly to GEO in Part 2.2

In [27]:
trait_subdir = "TCGA_Lung_Cancer_(LUNG)"
cohort = 'Xena'
# All the cancer traits in Xena are binary
trait_type = 'binary'
# Once a relevant cohort is found in Xena, we can generally assume the gene and clinical data are available
is_available = True

clinical_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.LUNG.sampleMap_LUNG_clinicalMatrix')
genetic_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.LUNG.sampleMap_HiSeqV2_PANCAN.gz')

In [28]:
import pandas as pd

clinical_data = pd.read_csv(clinical_data_file, sep='\t', index_col=0)
genetic_data = pd.read_csv(genetic_data_file, compression='gzip', sep='\t', index_col=0)
age_col = gender_col = None

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [ ]:
_, clinical_data_cols = check_rows_and_columns(clinical_data)

In [ ]:
clinical_data_cols[:10]

['ABSOLUTE_Ploidy',
 'ABSOLUTE_Purity',
 'AKT1',
 'ALK_translocation',
 'BRAF',
 'CBL',
 'CTNNB1',
 'Canonical_mut_in_KRAS_EGFR_ALK',
 'Cnncl_mt_n_KRAS_EGFR_ALK_RET_ROS1_BRAF_ERBB2_HRAS_NRAS_AKT1_MAP2',
 'EGFR']

Read all the column names in the clinical dataset, to find the columns that record information about age or gender.
Reference prompt:

In [ ]:
f'''
Below is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:
candidate_age_cols = [col_name1, col_name2, ...]
candidate_gender_cols = [col_name1, col_name2, ...]
If no columns match a criterion, please provide an empty list.

Column names:
{clinical_data_cols}
'''

"\nBelow is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:\ncandidate_age_cols = [col_name1, col_name2, ...]\ncandidate_gender_cols = [col_name1, col_name2, ...]\nIf no columns match a criterion, please provide an empty list.\n\nColumn names:\n['ABSOLUTE_Ploidy', 'ABSOLUTE_Purity', 'AKT1', 'ALK_translocation', 'BRAF', 'CBL', 'CTNNB1', 'Canonical_mut_in_KRAS_EGFR_ALK', 'Cnncl_mt_n_KRAS_EGFR_ALK_RET_ROS1_BRAF_ERBB2_HRAS_NRAS_AKT1_MAP2', 'EGFR', 'ERBB2', 'ERBB4', 'Estimated_allele_fraction_of_a_clonal_varnt_prsnt_t_1_cpy_pr_cll', 'Expression_Subtype', 'HRAS', 'KRAS', 'MAP2K1', 'MET', 'NRAS', 'PIK3CA', 'PTPN11', 'Pathology', 'Pathology_Updated', 'RET_translocation', 'ROS1_translocation', 'STK11', 'WGS_as_of_20120731_

In [ ]:
candidate_age_cols = ['age_at_initial_pathologic_diagnosis', 'days_to_birth', 'days_to_collection', 'days_to_death', 'days_to_initial_pathologic_diagnosis', 'days_to_last_followup', 'days_to_new_tumor_event_after_initial_treatment', 'stopped_smoking_year', 'year_of_initial_pathologic_diagnosis', 'year_of_tobacco_smoking_onset']

candidate_gender_cols = ['gender']

Choose a single column from the candidate columns that record age and gender information respectively.
If no column meets the requirement, keep 'age_col' or 'gender_col' to None

In [ ]:
preview_df(clinical_data[candidate_age_cols])

{'age_at_initial_pathologic_diagnosis': [70.0, 67.0, 79.0, 68.0, 66.0],
 'days_to_birth': [-25752.0, -24532.0, -29068.0, -24868.0, -24411.0],
 'days_to_collection': [nan, nan, nan, nan, nan],
 'days_to_death': [nan, nan, '121', nan, nan],
 'days_to_initial_pathologic_diagnosis': [0.0, 0.0, 0.0, 0.0, 0.0],
 'days_to_last_followup': ['0', '1523', nan, '607', '426'],
 'days_to_new_tumor_event_after_initial_treatment': [nan,
  nan,
  nan,
  334.0,
  183.0],
 'stopped_smoking_year': [nan, nan, nan, nan, 1987.0],
 'year_of_initial_pathologic_diagnosis': [2009.0,
  2007.0,
  2007.0,
  2009.0,
  2009.0],
 'year_of_tobacco_smoking_onset': [nan, nan, nan, nan, 1963.0]}

In [ ]:
age_col = 'age_at_initial_pathologic_diagnosis'

In [ ]:
preview_df(clinical_data[candidate_gender_cols])

{'gender': ['MALE', 'MALE', 'FEMALE', 'MALE', 'MALE']}

In [ ]:
gender_col = 'gender'

In [ ]:
selected_clinical_data = xena_select_clinical_features(clinical_data, TRAIT, age_col, gender_col)

In [ ]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

In [ ]:
merged_data = selected_clinical_data.join(genetic_data.T).dropna()
merged_data.head()

,Lung-Cancer,Age,Gender,ARHGEF10L,HIF3A,RNF17,RNF10,RNF11,RNF13,GTF2IP1,...,TULP2,NPY5R,GNGT2,GNGT1,TULP3,PTRF,BCL6B,GSTK1,SELP,SELS
sampleID,,,,,,,,,,,,,,,,,,,,,
TCGA-05-4244-01,1,70.0,1.0,-0.056592,2.128674,-0.531035,-0.185972,-0.190878,-0.11891,-0.659794,...,0.015022,-1.155117,0.528367,2.75061,0.428423,-0.639586,-0.195527,-0.324695,0.193167,-0.572812
TCGA-05-4249-01,1,67.0,1.0,-0.569692,-0.318126,-0.531035,0.237428,0.011822,-0.00611,1.018406,...,-0.031778,-0.393117,1.632467,2.33551,-0.145577,-0.997486,-0.276027,-1.219395,1.850967,-0.471212
TCGA-05-4250-01,1,79.0,0.0,-0.899992,-0.918126,-0.531035,-0.621472,-0.454178,0.58719,-0.198894,...,-0.748878,-1.173917,1.966367,-1.28139,-0.374877,0.959514,0.714373,0.112305,-0.532833,-0.003812
TCGA-05-4382-01,1,68.0,1.0,0.667508,3.895974,-0.221535,-0.367272,0.131822,0.97629,-0.494994,...,1.013722,-1.587117,0.359267,2.59051,0.806623,0.812114,0.367373,-0.494595,-1.062233,0.108588
TCGA-05-4384-01,1,66.0,1.0,-0.316892,2.380374,0.156865,0.232828,0.054822,0.27059,0.128606,...,-0.060978,-0.435417,1.111467,-1.28139,-0.128477,0.110214,1.469373,0.194605,1.404667,-0.290712


In [ ]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 1100 samples.


In [ ]:
is_trait_biased, merge_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Lung-Cancer', the least common label is '0' with 110 occurrences. This represents 10.00% of the dataset.
The distribution of the feature 'Lung-Cancer' in this dataset is fine.

Quartiles for 'Age':
  25%: 60.0
  50% (Median): 67.0
  75%: 73.0
Min: 38.0
Max: 90.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 443 occurrences. This represents 40.27% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



False

In [ ]:
merged_data.head()
if not is_trait_biased:
    merge_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

In [ ]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data)

A new JSON file was created at: ../../output1/Yuxuan/Lung-Cancer/cohort_info.json


## 2.2. The GEO dataset

In GEO, there may be one or multiple cohorts for a trait. Each cohort is identified by an accession number. We iterate over all accession numbers in the corresponding subdirectory, preprocess the cohort data, and save them to csv files.

In [2]:
dataset = 'GEO'
trait_subdir = "Psoriatic-Arthritis"

trait_path = os.path.join(DATA_ROOT, dataset, trait_subdir)
os.listdir(trait_path)

['GSE141934',
 'GSE100648',
 'GSE57376',
 'GSE57383',
 'GSE137634',
 'GSE142049',
 'GSE69371',
 'GSE69366',
 'GSE57405',
 'GSE57386',
 'GSE61281',
 'GSE69372',
 'GSE126017']

Repeat the below steps for all the accession numbers

In [3]:
cohort = accession_num = "GSE69372"
cohort_dir = os.path.join(trait_path, accession_num)
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)
soft_file, matrix_file

('/Users/wjt/Desktop/is_data/GEO/Psoriatic-Arthritis/GSE69372/GSE69372_family.soft.gz',
 '/Users/wjt/Desktop/is_data/GEO/Psoriatic-Arthritis/GSE69372/GSE69372_series_matrix.txt.gz')

### Inital filtering and clinical data preprocessing

In [4]:
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']

background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
print(background_info)

!Series_title	"Autoantibodies and nucleic acids skew complement consumption in systemic lupus erythematosus"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"


In [5]:
clinical_data.head()

,!Sample_geo_accession,GSM1698601,GSM1698602,GSM1698603,GSM1698604,GSM1698605,GSM1698606,GSM1698607,GSM1698608,GSM1698609,...,GSM1700323,GSM1700324,GSM1700325,GSM1700326,GSM1700327,GSM1700328,GSM1700329,GSM1700330,GSM1700331,GSM1700332
0,!Sample_characteristics_ch1,gender: Female,gender: Female,gender: Female,gender: Female,gender: Male,gender: Female,gender: Female,gender: Female,gender: Female,...,gender: Female,gender: Female,gender: Female,gender: Male,gender: Female,gender: Male,gender: Female,gender: Male,gender: Female,gender: Male
1,!Sample_characteristics_ch1,disease group (nhs-normal human serum; sle-sys...,disease group (nhs-normal human serum; sle-sys...,disease group (nhs-normal human serum; sle-sys...,disease group (nhs-normal human serum; sle-sys...,disease group (nhs-normal human serum; sle-sys...,disease group (nhs-normal human serum; sle-sys...,disease group (nhs-normal human serum; sle-sys...,disease group (nhs-normal human serum; sle-sys...,disease group (nhs-normal human serum; sle-sys...,...,disease group (nhs-normal human serum; sle-sys...,disease group (nhs-normal human serum; sle-sys...,disease group (nhs-normal human serum; sle-sys...,disease group (nhs-normal human serum; sle-sys...,disease group (nhs-normal human serum; sle-sys...,disease group (nhs-normal human serum; sle-sys...,disease group (nhs-normal human serum; sle-sys...,disease group (nhs-normal human serum; sle-sys...,disease group (nhs-normal human serum; sle-sys...,disease group (nhs-normal human serum; sle-sys...
2,!Sample_characteristics_ch1,age in years: 34,age in years: 31,age in years: 38,age in years: 30,age in years: 23,age in years: 43,age in years: 31,age in years: 48,age in years: 28,...,age in years: 29,age in years: 37,age in years: 54,age in years: 59,age in years: 32,age in years: 31,age in years: 47,age in years: 46,age in years: 23,age in years: 59
3,!Sample_characteristics_ch1,"printing batches (1,2): 1","printing batches (1,2): 1","printing batches (1,2): 1","printing batches (1,2): 1","printing batches (1,2): 1","printing batches (1,2): 1","printing batches (1,2): 1","printing batches (1,2): 1","printing batches (1,2): 1",...,"printing batches (1,2): 2","printing batches (1,2): 2","printing batches (1,2): 2","printing batches (1,2): 2","printing batches (1,2): 2","printing batches (1,2): 2","printing batches (1,2): 2","printing batches (1,2): 2","printing batches (1,2): 2","printing batches (1,2): 2"
4,!Sample_characteristics_ch1,slide no.: 1,slide no.: 1,slide no.: 1,slide no.: 1,slide no.: 1,slide no.: 1,slide no.: 1,slide no.: 1,slide no.: 1,...,slide no.: 30,slide no.: 30,slide no.: 30,slide no.: 30,slide no.: 30,slide no.: 30,slide no.: 30,slide no.: 30,slide no.: 30,slide no.: 30


In [6]:
clinical_data_unique = get_unique_values_by_row(clinical_data)
clinical_data_unique

{0: ['gender: Female', 'gender: Male'],
 1: ["disease group (nhs-normal human serum; sle-systemic lupus erythematosus ;uctd-undifferentiated connective tissue disease; sjs-sjörgen's syndrome;ssc-systemic sclerosis; psa-psoriatic arthritis): SLE",
  "disease group (nhs-normal human serum; sle-systemic lupus erythematosus ;uctd-undifferentiated connective tissue disease; sjs-sjörgen's syndrome;ssc-systemic sclerosis; psa-psoriatic arthritis): UCTD",
  "disease group (nhs-normal human serum; sle-systemic lupus erythematosus ;uctd-undifferentiated connective tissue disease; sjs-sjörgen's syndrome;ssc-systemic sclerosis; psa-psoriatic arthritis): SjS",
  "disease group (nhs-normal human serum; sle-systemic lupus erythematosus ;uctd-undifferentiated connective tissue disease; sjs-sjörgen's syndrome;ssc-systemic sclerosis; psa-psoriatic arthritis): NHS",
  "disease group (nhs-normal human serum; sle-systemic lupus erythematosus ;uctd-undifferentiated connective tissue disease; sjs-sjörgen's s

Analyze the metadata to determine data relevance and find ways to extract the clinical data.
Reference prompt:

In [7]:
f'''As a biomedical research team, we are selecting datasets to study the association between the human trait \'{TRAIT}\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:
1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)
2. For each of the traits \'{TRAIT}\', 'age', and 'gender', please address these points:
   (1) Is there human data available for this trait?
   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The key is an integer. The trait information might be explicitly recorded, or can be inferred from the field with some biomedical knowledge or understanding about the data collection process.
   (3) Choose an appropriate data type (either 'continuous' or 'binary') for each trait. Write a Python function to convert any given value of the trait to this data type. The function should handle inference about the trait value and convert unknown values to None.
   Name the functions 'convert_trait', 'convert_age', and 'convert_gender', respectively.

Background information about the dataset:
{background_info}

Sample characteristics dictionary (from "!Sample_characteristics_ch1", converted to a Python dictionary that stores the unique values for each field):
{clinical_data_unique}
'''

'As a biomedical research team, we are selecting datasets to study the association between the human trait \'Psoriatic-Arthritis\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:\n1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)\n2. For each of the traits \'Psoriatic-Arthritis\', \'age\', and \'gender\', please address these points:\n   (1) Is there human data available for this trait?\n   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. 

Understand and verify the answer from GPT, to assign values to the below variables. Assign None to the 'row_id' variables if relevant data row was not found.
Later we need to let GPT format its answer to automatically do these. But given the complexity of this step, let's grow some insight from the free-text answers for now.

In [8]:
is_gene_availabe = True
trait_row = 1
age_row = 2
gender_row = 0

trait_type = 'binary'

In [9]:
is_available = is_gene_availabe and (trait_row is not None)
if not is_available:
    save_cohort_info(cohort, JSON_PATH, is_available)
    print("This cohort is not usable. Please skip the following steps and jump to the next accession number.")

In [11]:
def convert_trait(tissue_type):
    """
    Convert tissue type to epilepsy presence (binary).
    Assuming epilepsy presence for 'Hippocampus' tissue.
    """
    if tissue_type == 'disease: Psoriatic Arthritis':
        return 1  # Epilepsy present
    else:
        return 0  # Epilepsy not present

def convert_age(age_string):
    """
    Convert age string to a numerical value.
    """
    if age_string.startswith('age in years: '):
        try:
            return int(age_string.split('age in years: ')[1].rstrip(''))  # Extracting numeric part
        except ValueError:
            return None  # In case of parsing error
    else:
        return None  # If age string does not follow expected format

def convert_gender(gender_string):
    """
    Convert gender string to a binary representation.
    Female -> 0, Male -> 1, Unknown -> None
    """
    if gender_string == 'gender: Female':
        return 0
    elif gender_string == 'gender: Male':
        return 1
    else:
        return None  # In case of unknown gender

# Example usage
example_tissue = 'tissue: Hippocampus'
example_age = 'age: 23y'
example_gender = 'gender: Male'

converted_trait = convert_trait(example_tissue)
converted_age = convert_age(example_age)
converted_gender = convert_gender(example_gender)

converted_trait, converted_age, converted_gender

(0, None, 1)

In [12]:
selected_clinical_data = geo_select_clinical_features(clinical_data, TRAIT, trait_row, convert_trait, age_row=age_row,
                                                      convert_age=convert_age, gender_row=gender_row,
                                                      convert_gender=convert_gender)
selected_clinical_data.head()

,GSM1698601,GSM1698602,GSM1698603,GSM1698604,GSM1698605,GSM1698606,GSM1698607,GSM1698608,GSM1698609,GSM1698610,...,GSM1700323,GSM1700324,GSM1700325,GSM1700326,GSM1700327,GSM1700328,GSM1700329,GSM1700330,GSM1700331,GSM1700332
Psoriatic-Arthritis,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Age,34,31,38,30,23,43,31,48,28,25,...,29,37,54,59,32,31,47,46,23,59
Gender,0,0,0,0,1,0,0,0,0,1,...,0,0,0,1,0,1,0,1,0,1


### Genetic data preprocessing and final filtering

In [13]:
genetic_data = get_genetic_data(matrix_file)
genetic_data.head()

,GSM1698601,GSM1698602,GSM1698603,GSM1698604,GSM1698605,GSM1698606,GSM1698607,GSM1698608,GSM1698609,GSM1698610,...,GSM1700323,GSM1700324,GSM1700325,GSM1700326,GSM1700327,GSM1700328,GSM1700329,GSM1700330,GSM1700331,GSM1700332
ID,,,,,,,,,,,,,,,,,,,,,
1,11021,12396,11083,10407,10263,16989,9986,14114,10414,14728,...,23158,14696,32353,15494,34519,18430,35063,19412,25291,17826
2,12252,13838,12692,10082,11503,14406,10123,12243,11415,16959,...,41863,11446,30365,11705,31850,14696,26042,14816,27171,18483
3,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
4,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
5,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [14]:
gene_row_ids = genetic_data.index[:20].tolist()
gene_row_ids

['1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20']

Check if the gene dataset requires mapping to get the gene symbols corresponding to each data row.

Reference prompt:

In [15]:
f'''
Below are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:
requires_gene_mapping = (True or False)

Row headers:
{gene_row_ids}
'''

"\nBelow are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:\nrequires_gene_mapping = (True or False)\n\nRow headers:\n['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20']\n"

If not required, jump directly to the gene normalization step

In [16]:
requires_gene_mapping = True

In [17]:
if requires_gene_mapping:
    gene_annotation = get_gene_annotation(soft_file)
    gene_annotation_summary = preview_df(gene_annotation)
    print(gene_annotation_summary)

{'ID': ['1', '2', '3', '4', '5'], 'SPOT_ID': ['BB', 'BB', 'PBS', 'hIgG{4}', 'hIgG{4}'], 'Full_Name': ['Fluorescent positioning spot', 'Fluorescent positioning spot', 'PBS', 'Human IgG Purified Immunoglobulin', 'Human IgG Purified Immunoglobulin'], 'Printed conc. (mg/ml)': [nan, nan, nan, '0.018519mg/ml', '0.018519mg/ml'], 'Supplier': [nan, nan, nan, 'Sigma', 'Sigma'], 'Cat.No.': [nan, nan, nan, 'I2511', 'I2511'], 'Lot.No.': [nan, nan, nan, '100M4830', '100M4830'], 'UniProt entry': [nan, nan, nan, nan, nan]}


Observe the first few cells in the ID column of the gene annotation dataframe, to find the names of columns that store the gene probe IDs and gene symbols respectively.
Reference prompt:

In [18]:
if requires_gene_mapping:
    print(f'''
    As a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:
    probe_name_key = 'key_name1'
    gene_name_key = 'key_name2'

    Gene annotation dictionary:
    {gene_annotation_summary}
    ''')


    As a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:
    probe_name_key = 'key_name1'
    gene_name_key = 'key_name2'

    Gene annotation dictionary:
    {'ID': ['1', '2', '3', '4', '5'], 'SPOT_ID': ['BB', 'BB', 'PBS', 'hIgG{4}', 'hIgG{4}'], 'Full_Name': ['Fluorescent positioning spot', 'Fluorescent positioning spot', 'PBS', 'Human IgG Purified Immunoglobulin', 'Human IgG Purified Immunoglobulin'], 'Printed conc. (mg/ml)': [nan, nan, nan, '0.018519mg/ml', '0.018519mg/ml'], 'Supplier': [nan, nan, nan, 'Sigma', 'Sigma'], 'Cat.No.': [nan, nan, nan, 'I2511', 'I2511'], 'Lot.No.': [nan, nan, nan, '100M4830', '100M4830'], 'UniProt entry': [nan, nan, nan, nan, nan]}
    


In [19]:
if requires_gene_mapping:
    probe_id_key = 'ID'
    gene_symb_key = 'SPOT_ID'
    gene_mapping = get_gene_mapping(gene_annotation, probe_id_key, gene_symb_key)
    genetic_data = apply_gene_mapping(genetic_data, gene_mapping)

In [20]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

In [21]:
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, genetic_data)
# The preprocessing runs through, which means is_available should be True
is_available = True

In [22]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 1700 samples.


In [23]:
is_trait_biased, merged_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Psoriatic-Arthritis', the least common label is '0.0' with 1700 occurrences. This represents 100.00% of the dataset.
The distribution of the feature 'Psoriatic-Arthritis' in this dataset is severely biased.

Quartiles for 'Age':
  25%: 34.0
  50% (Median): 43.0
  75%: 56.0
Min: 17.0
Max: 87.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 428 occurrences. This represents 25.18% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



True

In [24]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data, note='')

A new JSON file was created at: /Users/wjt/Desktop/is_data/v1/Lisa/Psoriatic-Arthritis/cohort_info.json


In [25]:
merged_data.head()
if not is_trait_biased:
    merged_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

### 3. Do regression & Cross Validation

In [26]:
from utils import *

In [27]:
best_cohort, ranked_df = filter_and_rank_cohorts(JSON_PATH, 'has_age')
best_cohort

In [97]:
ranked_df.head()

,cohort_id,is_usable,is_available,is_biased,has_age,has_gender,sample_size,note
2,GSE57405,True,True,False,True,True,111,
0,GSE57376,True,True,False,True,True,32,


In [98]:
ck=100
listq=[]
for df in pd.read_csv(os.path.join(OUTPUT_DIR, best_cohort+'.csv'),chunksize=ck):
    listq.append(df)
merged_data = pd.concat(listq)
#merged_data = pd.read_csv(os.path.join(OUTPUT_DIR, best_cohort+'.csv'))

In [99]:
merged_data.head()

,Psoriatic-Arthritis,Age,Gender,A1BG,A1CF,A2M,A2ML1,A4GALT,A4GNT,AAA1,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
0,0.0,54.0,1.0,5.8509,4.2783,4.90145,2.86270,5.4264,4.5460,3.56025,...,7.03480,7.0715,8.1001,6.93065,7.487683,4.4953,7.535633,8.82715,8.064000,8.61655
1,0.0,40.0,0.0,5.6841,4.2409,5.14220,2.69585,5.2648,4.5935,3.63820,...,6.64365,7.3864,6.6618,5.87660,7.223450,4.2151,8.334933,9.42815,7.260600,7.67455
2,0.0,64.0,1.0,5.8307,4.3646,5.06030,2.82075,5.5081,4.7189,3.59700,...,6.29695,6.7099,7.3957,6.12585,7.753533,4.4226,7.556633,9.60805,8.104433,8.34265
3,0.0,23.0,1.0,6.0151,4.3916,5.03605,2.82930,5.2146,4.6960,3.60185,...,6.68690,7.3324,7.5098,6.77530,7.633767,4.3393,7.814533,8.84590,7.788833,8.68620
4,0.0,60.0,0.0,6.0553,4.3842,4.88165,2.76585,5.3810,4.7212,3.55010,...,6.70660,6.9108,7.2595,6.56625,7.719517,3.7978,7.765867,9.09430,7.813967,8.24450


In [100]:
# If both age and gender features are available, select 'age' as the condition.
condition = 'Age'
# Remove the other condition to prevent interference.
merged_data = merged_data.drop(columns=['Gender'], errors='ignore').astype('float')

In [101]:
X = merged_data.drop(columns=[TRAIT, condition]).values
Y = merged_data[TRAIT].values
Z = merged_data[condition].values

Select the appropriate regression model depending on whether the dataset shows batch effect.

In [102]:
has_batch_effect = detect_batch_effect(X)
has_batch_effect

True

In [103]:

if has_batch_effect:
    model_constructor1 = VariableSelection
    model_params1 = {'modified': True, 'lamda': 3e-4}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}
else:
    model_constructor1 = Lasso
    model_params1 = {'alpha': 1.0, 'random_state': 42}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}

In [104]:
trait_type = 'binary'  # Remember to set this properly, either 'binary' or 'continuous'
cv_mean1, cv_std1 = cross_validation(X, Y, Z, model_constructor1, model_params1, target_type=trait_type)

alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
The cross-validation accuracy is 40.91% ± 13.79%


In [105]:
cv_mean2, cv_std2 = cross_validation(X, Y, Z, model_constructor2, model_params2, target_type=trait_type)

The cross-validation accuracy is 72.73% ± 14.94%


In [106]:
normalized_X, _ = normalize_data(X)
normalized_Z, _ = normalize_data(Z)

# Train regression model on the whole dataset to identify significant genes
model1 = ResidualizationRegressor(model_constructor1, model_params1)
model1.fit(normalized_X, Y, normalized_Z)

model2 = ResidualizationRegressor(model_constructor2, model_params2)
model2.fit(normalized_X, Y, normalized_Z)

alpha for Lasso: 0.0003


### 5. Discussion and report

In [107]:

feature_cols = merged_data.columns.tolist()
feature_cols.remove(TRAIT)

threshold = 0.05
interpret_result(model1, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=1)

Effect of the condition on the target variable:
Variable: Age
Coefficient: 0.1074
Found 78 genes with non-zero coefficients associated with the trait 'Psoriatic-Arthritis' conditional on the factor 'Age'. These genes are identified as significant based on the regression model.
 Variable  Coefficient  Absolute Coefficient
     MDN1     6.324511              6.324511
    CPT1C    -5.718712              5.718712
    NCOR1     5.032054              5.032054
   MAPK10    -4.819454              4.819454
   GALNT8     4.662987              4.662987
  RPL36AL     3.926921              3.926921
    NTAN1    -3.769901              3.769901
    PRKG2     3.741608              3.741608
   ZNF579    -3.537640              3.537640
   VPS13B     3.499023              3.499023
  CEACAM7     3.484398              3.484398
  KIR2DL4     3.348601              3.348601
    KNDC1    -3.186829              3.186829
     IFNE    -3.114752              3.114752
     TSHB     2.968850              2.968850
  

/home/yxcheng/AI4Science_gold_standard/stage1/YIjiang/../utils.py:469: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_regression_df['Absolute Coefficient'] = gene_regression_df['Coefficient'].abs()


In [108]:
interpret_result(model2, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=2)

Effect of the condition on the target variable:
Variable: Age
Coefficient: 0.1074
Found 1 significant genes associated with the trait 'Psoriatic-Arthritis' conditional on the factor 'Age', with corrected p-value < 0.05:
Variable  Coefficient      p_value  corrected_p_value
 COL18A1    -0.207399 1.019157e-07           0.001614


/home/yxcheng/AI4Science_gold_standard/stage1/YIjiang/../utils.py:478: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_regression_df.loc[:, 'corrected_p_value'] = corrected_p_values
